What is CrewAI?

CrewAI is an agent-based orchestration framework designed to let multiple AI agents work together like a real-world team. Instead of a single large language model doing everything, CrewAI organizes roles, responsibilities, and workflows—similar to how humans collaborate in workplaces.

Think of it as building a team of AI specialists, where each agent has:

A role (what they are)

A goal (what they must accomplish)

A backstory (context that influences behavior)

A specific task (work assigned to them)

Optionally, tools and memory

# Agents to Research and Write an Article

Foundational concepts of multi-agent systems and get an overview of the crewAI framework.

For running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [3]:
# !pip install crewai==0.28.8 crewai_tools==0.1.6


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

import warnings
warnings.filterwarnings('ignore')
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner


In [13]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    llm=llm,
	verbose=True
)

### Agent: Writer

In [14]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

### Agent: Editor

In [15]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [16]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [17]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [18]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 1 or 2 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [19]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2025-12-09 16:35:42,246 - 20884 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see here.

In [20]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer  
Final Answer: 

**Content Plan Document: Artificial Intelligence**

---

### 1. Latest Trends, Key Players, and Noteworthy News on Artificial Intelligence

**Trends:**
- **Generative AI:** The rise of AI models that can create text, images, and music, such as OpenAI's ChatGPT and DALL-E.
- **AI Ethics and Regulation:** Increasing focus on ethical AI use, with governments and organizations developing guidelines and regulations.
- **AI in Healthcare:** Advancements in AI for diagnostics, personalized medicine, and patient ca

- Display the results of your execution as markdown in the notebook.

In [21]:
from IPython.display import Markdown
Markdown(result)

```markdown
# Understanding Artificial Intelligence: Trends, Applications, and Future Implications

## I. Introduction

Artificial Intelligence (AI) has rapidly evolved from a niche area of research to a transformative force across various sectors. Its significance in today's world cannot be overstated, as AI technologies are reshaping industries, enhancing productivity, and influencing our daily lives. As we navigate through 2023, it is crucial to stay informed about the latest trends and developments in AI, as they hold the potential to redefine the future of work, healthcare, and beyond.

In this blog post, we will explore the latest trends in AI, highlight key players in the landscape, discuss noteworthy news, and examine the implications of AI on society. Whether you are a tech enthusiast, a business professional, or simply curious about AI, this article aims to provide valuable insights into the current state and future of artificial intelligence.

## II. Latest Trends in AI

### Generative AI and Its Applications

One of the most exciting trends in AI is the rise of generative AI, which refers to models capable of creating text, images, and even music. Notable examples include OpenAI's ChatGPT and DALL-E, which have garnered significant attention for their ability to generate human-like text and stunning visuals. This technology is revolutionizing content creation and opening new avenues for creativity and innovation across various fields.

Generative AI has applications in marketing, entertainment, and education, where it can assist in generating personalized content and enhancing user engagement. As these models continue to improve, we can expect to see even more sophisticated applications that blur the lines between human and machine-generated content.

### The Role of AI in Healthcare

AI's impact on healthcare is profound, with advancements in diagnostics, personalized medicine, and patient care management. AI algorithms are being used to analyze medical data, predict patient outcomes, and assist in surgical procedures. For instance, AI-driven tools can analyze medical images with remarkable accuracy, aiding radiologists in detecting anomalies that may be missed by the human eye.

Moreover, AI is facilitating personalized treatment plans by analyzing genetic information and patient history, leading to more effective healthcare solutions. As the healthcare industry continues to embrace AI, we can anticipate improved patient outcomes and more efficient healthcare delivery systems.

### Automation and Productivity Enhancements

The integration of AI in various industries is enhancing productivity and efficiency, particularly in manufacturing and logistics. AI-powered automation tools are streamlining operations, reducing human error, and optimizing supply chains. Companies are increasingly adopting AI solutions to manage inventory, forecast demand, and improve overall operational efficiency.

While automation presents significant benefits, it also raises concerns about job displacement. As AI systems take over repetitive tasks, workers may need to adapt by acquiring new skills to remain relevant in the evolving job market. Addressing these concerns is essential for a smooth transition into an AI-driven economy.

## III. Key Players in the AI Landscape

### Overview of Major Companies and Their Contributions

The AI landscape is populated by several key players, each contributing to the advancement of AI technologies. OpenAI is at the forefront with its generative models, pushing the boundaries of what AI can achieve. Google DeepMind is another significant player, focusing on AI research and applications across various fields, including healthcare and gaming.

IBM Watson has pioneered AI solutions tailored for business and healthcare, providing organizations with tools to harness the power of AI for data analysis and decision-making. Microsoft is also making strides by integrating AI into its products and services, particularly through Azure AI, which offers cloud-based AI solutions for businesses.

### Emerging Startups and Innovative Solutions

In addition to established companies, numerous startups are emerging with innovative AI solutions. These startups are often agile and focused on niche applications, driving competition and fostering creativity in the AI space. As investments in AI startups continue to grow, we can expect to see a wave of new technologies that challenge the status quo and offer unique solutions to pressing problems.

## IV. Noteworthy News and Developments

### Recent Breakthroughs and Their Implications

The AI field is witnessing remarkable breakthroughs, particularly in multimodal AI, which combines text, image, and audio processing. These advancements are enabling more sophisticated AI applications that can understand and generate content across different formats. For instance, AI systems can now create videos that incorporate both visual and auditory elements, enhancing storytelling and user engagement.

Moreover, major tech companies are investing heavily in AI research initiatives, signaling a commitment to advancing the field. These investments not only drive innovation but also raise questions about the ethical implications of AI technologies and their impact on society.

### Regulatory Discussions and Ethical Considerations

As AI technologies become more pervasive, discussions around ethics and regulation are intensifying. Governments and organizations are developing guidelines to ensure the responsible use of AI, addressing concerns related to data privacy, bias, and accountability. The challenge lies in balancing innovation with ethical considerations, as the rapid pace of AI development often outstrips regulatory frameworks.

It is essential for stakeholders, including policymakers, technologists, and the public, to engage in meaningful discussions about the ethical implications of AI. By fostering a collaborative approach, we can work towards a future where AI technologies are developed and deployed responsibly.

## V. Understanding the Impact of AI

### How AI is Changing Industries

AI is not just a technological advancement; it is a catalyst for change across industries. From finance to agriculture, AI is transforming how businesses operate and deliver value to customers. In finance, AI algorithms are used for fraud detection and risk assessment, while in agriculture, AI-driven tools optimize crop management and yield predictions.

The integration of AI into various sectors is leading to increased efficiency, reduced costs, and enhanced customer experiences. However, it is crucial to recognize that these changes also come with challenges, particularly in terms of workforce adaptation and ethical considerations.

### Addressing Concerns About Job Displacement and Ethics

One of the most pressing concerns surrounding AI is the potential for job displacement due to automation. While AI can enhance productivity, it may also render certain jobs obsolete. To address this issue, it is vital for businesses and governments to invest in reskilling and upskilling programs that prepare workers for the jobs of the future.

Additionally, ethical considerations surrounding AI use must be prioritized. Ensuring transparency, accountability, and fairness in AI systems is essential to build trust among users and mitigate potential biases. By addressing these concerns proactively, we can harness the benefits of AI while minimizing its risks.

## VI. Conclusion

In conclusion, understanding artificial intelligence is more important than ever as it continues to shape our world. From generative AI and healthcare advancements to the ethical implications of automation, staying informed about AI trends and developments is crucial for individuals and organizations alike. As we move forward, embracing AI responsibly will be key to unlocking its full potential and ensuring a positive impact on society.

## VII. Call to Action

We invite you to subscribe for updates on the latest AI trends and developments. Stay informed and engaged as we explore the fascinating world of artificial intelligence together. If you found this article insightful, please share it on social media to help spread awareness about the importance of understanding AI in today's rapidly changing landscape.
```

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)